In [126]:
!pip install s3fs

import boto3
import sagemaker
from sagemaker.session import Session
# from sagemaker.feature_store.feature_group import FeatureGroup

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)
# featuregroup_name = 'transactions-fg-manual-ingest'

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)
# feature_group = FeatureGroup(name=featuregroup_name, sagemaker_session=feature_store_session)

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [127]:
# record_identifier_feature_name = "ZIP"
# event_time_feature_name = "EventTime"

# change the bucket name to your desired bucket name 
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = ' hc-ds-pre-aca-tier'

print(default_s3_bucket_name)

# fg_test.create(
#     s3_uri=f"s3://{bucket}/{s3_folder}",
#     record_identifier_name=record_identifier_feature_name,
#     event_time_feature_name=event_time_feature_name,
#     role_arn=role,
#     enable_online_store=False)


sagemaker-us-east-1-915124832670


In [128]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print (role)

arn:aws:iam::915124832670:role/hc-sagemaker-default-execution-role


In [125]:
import pandas as pd

xls = pd.ExcelFile('Tier1CarrierCoverage.xlsx')
df = pd.read_excel(xls, 'Tier1', usecols="B,C,P")
df.columns = df.columns.str.replace(' ','_')
df.columns= df.columns.str.lower()
df.county = df.county.str.upper().str.strip()
df['county'] = df['county'].str.replace('DOÑA ANA','DONA ANA')
fip = pd.read_csv("zip_county_fips.csv")
fip['COUNTYNAME'] = fip['COUNTYNAME'].str.replace(' County','')
fip.columns = fip.columns.str.replace(' ','_')
fip.rename(columns={'COUNTYNAME': 'COUNTY', 'STCOUNTYFP':'FIP'}, inplace=True)
fip.columns = fip.columns.str.replace(' ','_')
fip.columns= fip.columns.str.lower()
fip.county = fip.county.str.strip().str.upper()
fip['county'] = fip['county'].str.replace(' PARISH','')
test = fip.groupby(['county','state']).max().reset_index()
test = test.drop(columns=['zip','classfp'])
final = df.merge(test,how='left',left_on=['county','state'],right_on=['county','state'], suffixes=('_ACA','_County'))
final

,state,county,total,fip
0,AL,BALDWIN,-3.0,1003
1,AL,GENEVA,-3.0,1061
2,AL,HENRY,-3.0,1067
3,AL,HOUSTON,-3.0,1069
4,AL,LIMESTONE,-3.0,1083
...,...,...,...,...
1636,WA,STEVENS,-3.5,53065
1637,WA,THURSTON,-3.5,53067
1638,WA,WALLA WALLA,-3.5,53071
1639,WA,WHITMAN,-2.5,53075


In [129]:
import time

current_time_sec = int(round(time.time()))
# append EventTime feature
final['EventTime'] = pd.Series([current_time_sec]*len(final), dtype="float64")

In [130]:
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype("str").astype("string")

# cast object dtype to string. The SageMaker Feature Store Python SDK will then map the string dtype to String feature type.
cast_object_to_string(final)

In [131]:
from time import gmtime, strftime, sleep

tier_feature_group_name = 'hc-ds-pre-aca-fip2tier' + strftime('%d-%H-%M-%S', gmtime())
# # fg_timestamp = strftime("%Y-%m-%d'T'%H:%M:%SZ", gmtime())
# df['EventTime'] = zip_feature_group_name

In [132]:
from sagemaker.feature_store.feature_group import FeatureGroup

tier_feature_group = FeatureGroup(name=tier_feature_group_name, sagemaker_session=feature_store_session)

In [133]:
record_identifier_feature_name = "fip"
event_time_feature_name = "EventTime"

In [134]:
tier_feature_group.load_feature_definitions(data_frame=final); # output is suppressed

In [ ]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")
tier_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}", #offline feature store bucket
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)
wait_for_feature_group_creation_complete(feature_group=tier_feature_group)

Waiting for Feature Group Creation


In [123]:
tier_feature_group.describe()


{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:915124832670:feature-group/hc-ds-pre-aca-mrktshr25-20-10-04',
 'FeatureGroupName': 'hc-ds-pre-aca-mrktshr25-20-10-04',
 'RecordIdentifierFeatureName': 'state',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'state', 'FeatureType': 'String'},
  {'FeatureName': 'total_individual_market__enrollment',
   'FeatureType': 'Integral'},
  {'FeatureName': 'largest_insurer__insurer_name', 'FeatureType': 'String'},
  {'FeatureName': 'largest_insurer__enrollment', 'FeatureType': 'Integral'},
  {'FeatureName': 'largest_insurer__market_share',
   'FeatureType': 'Fractional'},
  {'FeatureName': 'second_largest_insurer__insurer_name',
   'FeatureType': 'String'},
  {'FeatureName': 'second_largest_insurer__enrollment',
   'FeatureType': 'Integral'},
  {'FeatureName': 'second_largest_insurer__market_share',
   'FeatureType': 'Fractional'},
  {'FeatureName': 'third_largest_insurer__insurer_name',
   'FeatureType': 'String'},
 